# Introduction
Recently, I studied Switzerland's international trade based on data from the UN Commodities Trade Database (https://comtradeplus.un.org/), which gathers information about countries, trade flows, modes of transport and types of commodities during a certain period of time. At some point, I was curious about the fact that there are not many countries with which Switzerland have a substantial deficit in trade. By substantial, I mean more or equal to 20% of the overall trade per year. The 20% threshold was adopted based on the lower limit of the new customs tariffs introduced in 2025 by the USA.

Besides Germany (with which Swiss have average deficit of 10% and 12.5 bln USD), most of the "deficit-makers" do not make more than 4.1 bln USD of deficit per year, which is around 1% of the total imported goods to Switzerland. What is more interesting is that very quickly a substantial deficit becomes almost entirely the value of imports from several countries (where imports constitute 95-100% of bilateral trade). So, a vast part of the so-called deficit-makers only sell something to Switzerland, without buying anything in return.

What is it that these countries have and Switzerland does not? What kind of advantage do they have over Switzerland? And is it really their or actually Switzerland's advantage? What brings together countries like Uzbekistan, South Africa, Ghana, Burkina Faso, the Ivory Coast, Peru or Kazakhstan with Switzerland?

# Preview

## Combined trading fact table of Switzerland and dim tables

Preview of trading fact table for Switzerland (ca. 200k rows) :

In [5]:
SELECT *
FROM ch_2020_24.csv
LIMIT 5;

,trade_year,trade_flow,partner_code,partner_iso,commodities_code,trans_code,trans_name,trade_value
0,2020,Export,533,ABW,27,1000,Air,927.470
1,2020,Export,533,ABW,33,1000,Air,2229.127
2,2020,Export,533,ABW,38,1000,Air,70.360
3,2020,Import,533,ABW,88,1000,Air,203125.386
4,2020,Import,533,ABW,91,1000,Air,44424.015


Preview of dim table with names of countries:

In [6]:
SELECT *
FROM countries.csv
LIMIT 5

,partner_code,partner_iso,partner_name,partner_note
0,4,AFG,Afghanistan,Afghanistan
1,472,_AC,"Africa CAMEU region, nes","Africa CAMEU region, not elsewhere specified"
2,248,ALA,Aland Islands,None
3,8,ALB,Albania,Albania
4,12,DZA,Algeria,Algeria


Preview of dim table with names of commodities

In [7]:
SELECT *
FROM commodities.csv
LIMIT 5

,commodities_code,commodities_name
0,9999,Commodities not specified according to kind
1,99,Commodities not specified according to kind
2,9706,Antiques; of an age exceeding one hundred years
3,9705,Collections and collectors' pieces; of archaeo...
4,9704,"Stamps, postage or revenue; stamp-postmarks, f..."


# Exploratory Data Analysis

Table showing all the fields:

In [8]:
SELECT *
FROM ch_2020_24.csv AS a
LEFT JOIN countries.csv AS b
		USING (partner_code)
LEFT JOIN commodities.csv AS c
		ON a.commodities_code = c.commodities_code
LIMIT 5

,trade_year,trade_flow,partner_code,partner_iso,commodities_code,trans_code,trans_name,trade_value,partner_iso_1,partner_name,partner_note,commodities_code_1,commodities_name
0,2020,Export,533,ABW,27,1000,Air,927.470,ABW,Aruba,Aruba,27,"Mineral fuels, mineral oils and products of th..."
1,2020,Export,533,ABW,33,1000,Air,2229.127,ABW,Aruba,Aruba,33,"Essential oils and resinoids; perfumery, cosme..."
2,2020,Export,533,ABW,38,1000,Air,70.360,ABW,Aruba,Aruba,38,Chemical products n.e.c.
3,2020,Import,533,ABW,88,1000,Air,203125.386,ABW,Aruba,Aruba,88,"Aircraft, spacecraft, and parts thereof"
4,2020,Import,533,ABW,91,1000,Air,44424.015,ABW,Aruba,Aruba,91,Clocks and watches and parts thereof


# Focus on trading partners

**To grasp the scale of the topic:**

**- all traded value per year** (bln $)

**- how big is 1% of that yearly trades** (bln $)

**- how much % the traded value grew compared to the previous year?**

**- how big is the top 10 countries compared to the overall trade per year?**

In [9]:
SELECT
	*,
	trade_value_bln_usd/100 AS one_percent_yearly_trade,
	ROUND((trade_value_bln_usd * 100) / (LAG(trade_value_bln_usd,1) OVER (ORDER BY trade_year)),2) - 100 AS yearly_growth_percent
FROM (
	SELECT
		trade_year,
		ROUND((SUM(trade_value)/1000000000),2) as trade_value_bln_usd,
	FROM ch_2020_24.csv AS a
	GROUP BY trade_year
)
ORDER BY trade_year

,trade_year,trade_value_bln_usd,one_percent_yearly_trade,yearly_growth_percent
0,2020,608.98,6.0898,NaN
1,2021,703.13,7.0313,15.46
2,2022,756.29,7.5629,7.56
3,2023,785.30,7.8530,3.84
4,2024,818.21,8.1821,4.19


**Swiss trade balance of each year:**

In [10]:
WITH import AS (
	SELECT
		trade_year,
		trade_flow,
		ROUND((SUM(trade_value)/1000000000),2) as import_value_bln_usd,
	FROM ch_2020_24.csv AS a
	WHERE trade_flow = 'Import'
	GROUP BY trade_year, trade_flow
),
export AS(
	SELECT
		trade_year,
		trade_flow,
		ROUND((SUM(trade_value)/1000000000),2) as export_value_bln_usd,
	FROM ch_2020_24.csv AS a
	WHERE trade_flow = 'Export'
	GROUP BY trade_year, trade_flow
),
total_year AS (
	SELECT
		trade_year,
		ROUND((SUM(trade_value)/1000000000),2) as total_trade_value_bln_usd,
	FROM ch_2020_24.csv AS a
	GROUP BY trade_year
)

SELECT
	b.trade_year,
	export_value_bln_usd,
	import_value_bln_usd,
	total_trade_value_bln_usd,
	export_value_bln_usd - import_value_bln_usd AS surplus_deficit_bln,
	ROUND(((export_value_bln_usd - import_value_bln_usd) *100/total_trade_value_bln_usd),2) AS percent_surplus
FROM import AS a
INNER JOIN export AS b
	USING(trade_year)
INNER JOIN total_year AS c
	USING(trade_year)
ORDER BY trade_year

,trade_year,export_value_bln_usd,import_value_bln_usd,total_trade_value_bln_usd,surplus_deficit_bln,percent_surplus
0,2020,318.58,290.40,608.98,28.18,4.63
1,2021,379.77,323.36,703.13,56.41,8.02
2,2022,400.06,356.24,756.29,43.82,5.79
3,2023,419.92,365.37,785.30,54.55,6.95
4,2024,446.84,371.36,818.21,75.48,9.23


**Top 10 countries for each year,**

**Their trade value and overall trade value by year,**

**Each country's part in the overall trade value by year:**

In [11]:
WITH trade_rank AS (
SELECT
	a.trade_year,
	c.partner_name,
	RANK() OVER(PARTITION BY a.trade_year ORDER BY SUM(a.trade_value) DESC) AS rank,
	ROUND((SUM(trade_value)/1000000000),2) as trade_value_country_bln_usd,
	b.trade_value_bln_usd,
	ROUND(((SUM(trade_value)/1000000000) *100 / b.trade_value_bln_usd),2) AS percent_of_whole
FROM ch_2020_24.csv AS a
LEFT JOIN traded_value_per_year AS b
	USING(trade_year)
LEFT JOIN countries.csv AS c
	ON a.partner_code = c.partner_code
GROUP BY a.trade_year, b.trade_value_bln_usd, c.partner_name)

SELECT *
FROM trade_rank
WHERE rank <=10
ORDER BY trade_year, rank

,trade_year,partner_name,rank,trade_value_country_bln_usd,trade_value_bln_usd,percent_of_whole
0,2020,Germany,1,105.57,608.98,17.34
1,2020,USA,2,94.04,608.98,15.44
2,2020,Italy,3,39.15,608.98,6.43
3,2020,China,4,35.10,608.98,5.76
4,2020,France,5,33.55,608.98,5.51
5,2020,United Kingdom,6,33.44,608.98,5.49
6,2020,"China, Hong Kong SAR",7,18.40,608.98,3.02
7,2020,Austria,8,17.14,608.98,2.81
8,2020,Spain,9,15.49,608.98,2.54
9,2020,India,10,13.50,608.98,2.22


**What part of the total trade value are top 10 countries?**

In [12]:
SELECT
	trade_year,
	SUM(trade_value_country_bln_usd) AS top10,
	trade_value_bln_usd,
	ROUND((SUM(trade_value_country_bln_usd) *100/trade_value_bln_usd),2) AS top10_to_whole
FROM top_10
GROUP BY trade_year, trade_value_bln_usd
ORDER BY trade_year

,trade_year,top10,trade_value_bln_usd,top10_to_whole
0,2020,405.38,608.98,66.57
1,2021,482.06,703.13,68.56
2,2022,487.78,756.29,64.50
3,2023,492.10,785.30,62.66
4,2024,537.84,818.21,65.73


# Conclusions Trading Partners

## **1. Dynamics of Trade Growth**

**Volatile Growth Recovery:** While the 5-year average growth rate stands at 7.8%, the data reveals a significant post-pandemic spike of +15.5% (2020–2021).

**Stabilization Trend:** In the most recent period (2023–2024), trade growth has stabilized at a sustainable 4.0% YoY.

Most likely, the 2021 outlier was a "rebound effect" rather than a long-term structural shift.

## **2. Trade Surplus & Regulatory Environment**

**Expansion of Trade Margin:** Switzerland significantly strengthened its trade position, nearly doubling its surplus from 4.6% (2020) to 9.2% (2024). Nominally, the surplus surged from 28.2 bilion USD to 75.5 bilion USD.

## **3. Market Concentration and Counterparty Risk**

**High Dependency on Key Partners:** Trade remains highly concentrated, with the Top 10 partners accounting for 63-69% of total volume.

**The "G2" Dominance:** Germany and the USA alone account for ~30% of Switzerland’s total trade value.

Such high concentration makes the Swiss economy particularly sensitive to the trade policies and economic health of these two specific nations.

## **4. Notes for further research:**

To move beyond aggregate figures:

**Commodity Deep-Dive:** Identifying high-growth categories within the German and American trade flows.

**Logistics & Infrastructure:** Analyzing the primary modes of transport to identify the most critical trade corridors.

# Focus on commodities

**Like with the countries, it is interesting to see top trading commodities and how they relate to flow direction (Import/Export)**

**Top 20 groups of commodities ordered by the biggest trade value and by year - only for Export:**

In [13]:
SELECT
	trade_year,
	a.commodities_code,
	commodities_name,
	ROUND((SUM(trade_value)/1000000000),2) as trade_value_bln_usd,
	RANK() OVER(PARTITION BY a.trade_year ORDER BY SUM(a.trade_value) DESC) AS rank
FROM ch_2020_24.csv AS a
LEFT JOIN commodities.csv AS c
	USING(commodities_code)
WHERE trade_flow = 'Export'
GROUP BY trade_year, a.commodities_code, commodities_name
ORDER BY rank, trade_year
LIMIT 100

,trade_year,commodities_code,commodities_name,trade_value_bln_usd,rank
0,2020,30,Pharmaceutical products,88.35,1
1,2021,71,"Natural, cultured pearls; precious, semi-preci...",105.75,1
2,2022,71,"Natural, cultured pearls; precious, semi-preci...",119.64,1
3,2023,71,"Natural, cultured pearls; precious, semi-preci...",126.98,1
4,2024,71,"Natural, cultured pearls; precious, semi-preci...",136.94,1
...,...,...,...,...,...
95,2020,38,Chemical products n.e.c.,1.26,20
96,2021,72,Iron and steel,1.67,20
97,2022,72,Iron and steel,1.80,20
98,2023,22,"Beverages, spirits and vinegar",1.73,20


**Top 20 groups of commodities ordered by the biggest trade value and by year - only for Import:**

In [14]:
SELECT
	trade_year,
	a.commodities_code,
	commodities_name,
	ROUND((SUM(trade_value)/1000000000),2) as trade_value_bln_usd,
	RANK() OVER(PARTITION BY a.trade_year ORDER BY SUM(a.trade_value) DESC) AS rank
FROM ch_2020_24.csv AS a
LEFT JOIN commodities.csv AS c
	USING(commodities_code)
WHERE trade_flow = 'Import'
GROUP BY trade_year, a.commodities_code, commodities_name
ORDER BY rank, trade_year
LIMIT 100

,trade_year,commodities_code,commodities_name,trade_value_bln_usd,rank
0,2020,71,"Natural, cultured pearls; precious, semi-preci...",103.80,1
1,2021,71,"Natural, cultured pearls; precious, semi-preci...",109.87,1
2,2022,71,"Natural, cultured pearls; precious, semi-preci...",117.70,1
3,2023,71,"Natural, cultured pearls; precious, semi-preci...",121.97,1
4,2024,71,"Natural, cultured pearls; precious, semi-preci...",125.62,1
...,...,...,...,...,...
95,2020,38,Chemical products n.e.c.,2.08,20
96,2021,64,Footwear; gaiters and the like; parts of such ...,2.33,20
97,2022,38,Chemical products n.e.c.,2.36,20
98,2023,44,Wood and articles of wood; wood charcoal,2.25,20


**Top 10 commodities by average year**

Comparison of Import to Export of 10 groups of commodities by average (yearly) trade value,

And the outcome: surplus of deficit

In [15]:
-- AVG of import/export by commodities_code counted as SUM divided by 5y to simplify the code.
WITH export_goods AS (
	SELECT
		a.commodities_code,
		commodities_name,
		ROUND(((SUM(trade_value)/5)/1000000000),2) as avg_trade_value_bln_usd,
		RANK() OVER(PARTITION BY a.trade_flow ORDER BY (SUM(trade_value)/5) DESC) AS rank_export
	FROM ch_2020_24.csv AS a
	LEFT JOIN commodities.csv AS c
		USING(commodities_code)
	WHERE trade_flow = 'Export'
	GROUP BY trade_flow, a.commodities_code, commodities_name
),
	
import_goods AS (
	SELECT
		a.commodities_code,
		commodities_name,
		ROUND(((SUM(trade_value)/5)/1000000000),2) as avg_trade_value_bln_usd,
		RANK() OVER(PARTITION BY a.trade_flow ORDER BY SUM(a.trade_value)/5 DESC) AS rank_import
	FROM ch_2020_24.csv AS a
	LEFT JOIN commodities.csv AS c
		USING(commodities_code)
	WHERE trade_flow = 'Import'
	GROUP BY trade_flow, a.commodities_code, commodities_name
)

SELECT
	e.commodities_code,
	SUBSTRING(e.commodities_name,0,80) AS commodities_name,
	rank_export,
	rank_import,
	e.avg_trade_value_bln_usd AS export_value_bln_usd,
	i.avg_trade_value_bln_usd AS import_value_bln_usd,
	e.avg_trade_value_bln_usd + i.avg_trade_value_bln_usd AS avg_traded_volume,
	e.avg_trade_value_bln_usd - i.avg_trade_value_bln_usd AS avg_surplus_or_deficit
FROM export_goods AS e
FULL OUTER JOIN import_goods AS i
	USING (commodities_code)
ORDER BY avg_traded_volume desc
LIMIT 10

,commodities_code,commodities_name,rank_export,rank_import,export_value_bln_usd,import_value_bln_usd,avg_traded_volume,avg_surplus_or_deficit
0,71,"Natural, cultured pearls; precious, semi-preci...",1,1,115.02,115.79,230.81,-0.77
1,30,Pharmaceutical products,2,2,99.46,51.22,150.68,48.24
2,84,"Machinery and mechanical appliances, boilers, ...",5,3,24.10,21.44,45.54,2.66
3,29,Organic chemicals,3,8,34.73,8.33,43.06,26.40
4,85,Electrical machinery and equipment and parts t...,7,5,13.98,17.26,31.24,-3.28
5,91,Clocks and watches and parts thereof,4,14,25.56,3.55,29.11,22.01
6,90,"Optical, photographic, cinematographic, measur...",6,7,19.01,9.24,28.25,9.77
7,87,Vehicles; other than railway or tramway rollin...,13,4,2.87,17.49,20.36,-14.62
8,27,"Mineral fuels, mineral oils and products of th...",9,6,5.52,13.11,18.63,-7.59
9,39,Plastics and articles thereof,8,9,5.91,7.66,13.57,-1.75


# Conclusions Commodities

## 1. The "Big Two" Dominance

**Switzerland’s trade is characterized by extreme sectoral concentration. Two categories alone account for over 50% of total trade volume:**

- **Precious Metals & Stones (over 30%):** Operates on a 1:1 Import-to-Export ratio. This makes Switzerland a "Pass-through Hub" rather than a consumer.

- **Pharmaceutical Products:** Exports outpace imports 2:1, with the biggest nominal surplus of 48.24 billion USD out of 150.58 billion USD of cumulated trade with this products.

- **Fragmented Rest:** Beyond these two giants, the trade landscape becomes extremely fragmented. The third-largest category accounts for only 6% of total trade.

## 2. High-Margin Export Clusters

The analysis identifies four sectors with massive trade surpluses, indicating high global competitiveness and "Swiss-made" brand label:

- **Pharmaceutical Products** (2nd biggest trade value): 32% surplus to trade value

- **Watchmaking:** Dominance with a 76% surplus to trade value ratio.

- **Organic Chemicals:** High-value manufacturing with a 60% surplus.

- **Precision Instruments (Medical/Optical):** A 34% surplus, reflecting the specialized nature of the Swiss tech sector.

## 3. Strategic Deficits: The Consumer & Industrial Gap

- **Vehicles & Transport:** This category represents the largest structural deficit among the Top 10 (85% of traded value consists of imports).

# Import/Export, Deficit and Surplus by Trading Partner

Looking into aggregation values of the biggest partners:
- Ranking in Import/Export
- Average Trade Value (per year)
- Max and Min values per year for Export and Import with each country
- What is the total volume of trade, and with whom is the biggest?
- How does the average deficit/surplus look in relation to the average yearly volume of the trade?
- **What is the ratio between the yearly trade volume and surplus/deficit across partners of Switzerland? Who gains more on the trade exchange?**

**With whom did Switzerland trade the most over the last 5 years?**

**Did it end with a surplus or a deficit and how big?**

**General table with all trading partners**

How does average year in Swiss trade looks like?

The table shows each trading partner, their traded volume and outcome (deficit or surplus).

In [16]:
WITH imports_avg_year AS
	(
	SELECT
		partner_code,
		RANK() OVER(ORDER BY AVG(trade_value) DESC) AS rank_import,
		ROUND((MIN(trade_value)/1000000000),2) AS min_import_bln,
		ROUND((MAX(trade_value)/1000000000),2) AS max_import_bln,
		ROUND((AVG(trade_value)/1000000000),2) AS avg_import_bln
	FROM
		( -- subquery - selecting aggregated total trade_value per year per country
		SELECT
			trade_year,
			partner_code,
			SUM(trade_value) AS trade_value
		FROM ch_2020_24.csv
		WHERE trade_flow = 'Import'
		GROUP BY trade_year, partner_code
		)
	GROUP BY partner_code
	),

exports_avg_year AS
	(
	SELECT
		partner_code,
		RANK() OVER(ORDER BY AVG(trade_value) DESC) AS rank_export,
		ROUND((MIN(trade_value)/1000000000),2) AS min_export_bln,
		ROUND((MAX(trade_value)/1000000000),2) AS max_export_bln,
		ROUND((AVG(trade_value)/1000000000),2) AS avg_export_bln
	FROM
		( -- subquery - selecting aggregated total trade_value per year per country
		SELECT
			trade_year,
			partner_code,
			SUM(trade_value) AS trade_value
		FROM ch_2020_24.csv
		WHERE trade_flow = 'Export'
		GROUP BY trade_year, partner_code
		)
	GROUP BY partner_code
	)

SELECT
	c.partner_name,
	rank_export,
	min_export_bln,
	max_export_bln,
	avg_export_bln,
	rank_import,
	min_import_bln,
	max_import_bln,
	avg_import_bln,
	avg_export_bln + avg_import_bln AS avg_volume_year_bln,
	avg_export_bln - avg_import_bln AS avg_surplus_deficit_year_bln,
	ROUND((avg_export_bln - avg_import_bln) * 100 / (avg_export_bln + avg_import_bln),1) AS percent_surplus_deficit_to_volume
FROM exports_avg_year as e
FULL OUTER JOIN imports_avg_year as i
	ON e.partner_code = i.partner_code
LEFT JOIN countries.csv AS c
	ON e.partner_code = c.partner_code
ORDER BY avg_volume_year_bln desc

,partner_name,rank_export,min_export_bln,max_export_bln,avg_export_bln,rank_import,min_import_bln,max_import_bln,avg_import_bln,avg_volume_year_bln,avg_surplus_deficit_year_bln,percent_surplus_deficit_to_volume
0,Germany,2,48.96,55.24,51.99,1,56.61,69.64,64.55,116.54,-12.56,-10.8
1,USA,1,62.67,73.88,67.52,2,20.89,39.24,29.54,97.06,37.98,39.1
2,China,3,17.44,45.16,36.42,5,17.67,21.55,19.96,56.38,16.46,29.2
3,Italy,4,15.48,28.03,23.17,3,23.67,30.37,26.26,49.43,-3.09,-6.3
4,France,5,16.47,21.09,19.59,4,17.08,22.99,20.42,40.01,-0.83,-2.1
...,...,...,...,...,...,...,...,...,...,...,...,...
226,Cocos Isds,226,0.00,0.00,0.00,<NA>,NaN,NaN,NaN,NaN,NaN,NaN
227,Nauru,227,0.00,0.00,0.00,<NA>,NaN,NaN,NaN,NaN,NaN,NaN
228,Western Sahara,228,0.00,0.00,0.00,<NA>,NaN,NaN,NaN,NaN,NaN,NaN
229,Christmas Isds,230,0.00,0.00,0.00,<NA>,NaN,NaN,NaN,NaN,NaN,NaN


**What is total Switzerland's deficit?**

In [17]:
-- AVG of import/export by commodities_code counted as SUM divided by 5y to simplify the code.

WITH imports_avg_year AS
		(
		SELECT
			partner_code,
			ROUND(((SUM(trade_value)/5)/1000000000),2) AS avg_import_bln
		FROM ch_2020_24.csv
		WHERE trade_flow = 'Import'
		GROUP BY partner_code
		),

exports_avg_year AS
		(
		SELECT
			partner_code,
			ROUND(((SUM(trade_value)/5)/1000000000),2) AS avg_export_bln
		FROM ch_2020_24.csv
		WHERE trade_flow = 'Export'
		GROUP BY partner_code
		)

SELECT
	SUM(avg_deficit_year_bln) AS total_deficit_bln,
	COUNT (DISTINCT partner_code) AS count_countries
FROM ( -- subquery - selecting aggregated mean yearly deficit per country
	SELECT 
		avg_export_bln - avg_import_bln AS avg_deficit_year_bln,
		partner_code
	FROM imports_avg_year
	LEFT JOIN exports_avg_year
		USING (partner_code)
	WHERE avg_deficit_year_bln < 0
)


,total_deficit_bln,count_countries
0,-64.43,64


**Checking only countries with which Switzerland had a significant deficit (at least 20% of the total trade value) on average year and with whom overall yearly trade value was bigger or equal to 100 mln USD.**

Note: The 20% threshold was adopted based on the lower limit of new customs tariffs introduced in April 2025 by the USA.

In [18]:
SELECT *,
	ROUND(avg_surplus_deficit_year_bln *100/-64.43) AS percent_deficit
FROM avg_agg_imports_exports
WHERE avg_volume_year_bln >= 0.1
	AND percent_surplus_deficit_to_volume <= -20
ORDER BY avg_volume_year_bln DESC

,partner_name,rank_export,min_export_bln,max_export_bln,avg_export_bln,rank_import,min_import_bln,max_import_bln,avg_import_bln,avg_volume_year_bln,avg_surplus_deficit_year_bln,percent_surplus_deficit_to_volume,percent_deficit
0,Ireland,31,1.31,1.96,1.54,15,3.86,5.31,4.49,6.03,-2.95,-48.9,5.0
1,South Africa,46,0.55,0.72,0.64,20,2.42,6.37,4.11,4.75,-3.47,-73.1,5.0
2,Uzbekistan,80,0.07,0.22,0.12,19,0.01,8.18,4.20,4.32,-4.08,-94.4,6.0
3,Ghana,104,0.03,0.07,0.04,22,2.45,4.28,3.32,3.36,-3.28,-97.6,5.0
4,Burkina Faso,158,0.00,0.01,0.01,25,2.99,3.38,3.16,3.17,-3.15,-99.4,5.0
5,Viet Nam,51,0.50,0.61,0.55,31,1.88,2.88,2.16,2.71,-1.61,-59.4,2.0
6,Indonesia,53,0.36,0.53,0.44,29,1.70,2.78,2.25,2.69,-1.81,-67.3,3.0
7,Peru,78,0.13,0.15,0.14,27,1.82,3.08,2.43,2.57,-2.29,-89.1,4.0
8,Kazakhstan,56,0.26,0.51,0.37,34,0.08,3.75,1.96,2.33,-1.59,-68.2,2.0
9,Finland,45,0.63,0.73,0.68,39,1.17,1.86,1.59,2.27,-0.91,-40.1,1.0


**What is total deficit made by 38 selected countries, the biggest so-called "deficit-makers"?**

In [19]:
SELECT
	SUM(avg_surplus_deficit_year_bln) AS filtered_deficit_total,
	ROUND((SUM(avg_surplus_deficit_year_bln)*100/-64.43),0) AS percent_of_total,
	COUNT(partner_name) AS count_countries
FROM avg_agg_imports_exports
WHERE avg_volume_year_bln >= 0.1
	AND percent_surplus_deficit_to_volume <= -20

,filtered_deficit_total,percent_of_total,count_countries
0,-39.71,62.0,38


Negative trade imbalance with many of these countries goes up to almost 100% of bilateral traded value.

**In which areas (commodities' groups) these countries have advantage over Switzerland?**

**Table below shows a subset with states, with which Switzerland had the biggest deficit on average year (at least 80% of the total trade value) and with whom the overall yearly trade value was bigger or equal to 0.1bln $**

In [20]:
SELECT
	b.partner_name,
	a.commodities_code,
	c.commodities_name,
	RANK() OVER(PARTITION BY partner_name ORDER BY AVG(trade_value) DESC) AS rank_import,
	ROUND((MIN(trade_value)/1000000000),2) AS min_import_bln,
	ROUND((MAX(trade_value)/1000000000),2) AS max_import_bln,
	ROUND((AVG(trade_value)/1000000000),2) AS avg_import_bln
FROM
	( -- subquery - selecting aggregated total trade_value per year per country and commodities_code
	SELECT
		trade_year,
		partner_code,
		commodities_code,
		SUM(trade_value) AS trade_value
	FROM ch_2020_24.csv
	WHERE trade_flow = 'Import'
	GROUP BY trade_year, partner_code, commodities_code
		) AS a
LEFT JOIN countries.csv AS b
		USING (partner_code)
LEFT JOIN commodities.csv AS c
		ON a.commodities_code = c.commodities_code
WHERE partner_name IN
	(
	SELECT partner_name
	FROM 
	-- subquery 'deficit-makers' - (subset) with countries, with which Switzerland had the biggest deficit on average year (at least 80% of the total trade value) and with whom the overall yearly trade value was bigger or equal to 0.1bln $ 
		(
			SELECT *
			FROM avg_agg_imports_exports
			WHERE avg_volume_year_bln >= 0.1
				AND percent_surplus_deficit_to_volume <= -80
		)
	)
GROUP BY partner_name, a.commodities_code, commodities_name
ORDER BY avg_import_bln DESC
LIMIT 40

,partner_name,commodities_code,commodities_name,rank_import,min_import_bln,max_import_bln,avg_import_bln
0,Uzbekistan,71,"Natural, cultured pearls; precious, semi-preci...",1,0.01,8.18,4.20
1,Ghana,71,"Natural, cultured pearls; precious, semi-preci...",1,2.35,4.16,3.22
2,Burkina Faso,71,"Natural, cultured pearls; precious, semi-preci...",1,2.98,3.38,3.16
3,Peru,71,"Natural, cultured pearls; precious, semi-preci...",1,1.71,2.90,2.28
4,Ivory Coast,71,"Natural, cultured pearls; precious, semi-preci...",1,0.85,2.71,1.53
5,Suriname,71,"Natural, cultured pearls; precious, semi-preci...",1,1.09,1.32,1.19
6,Mali,71,"Natural, cultured pearls; precious, semi-preci...",1,0.12,1.50,1.05
7,Mongolia,71,"Natural, cultured pearls; precious, semi-preci...",1,0.84,1.11,0.93
8,Dominican Rep.,71,"Natural, cultured pearls; precious, semi-preci...",1,0.66,1.19,0.91
9,Kyrgyzstan,71,"Natural, cultured pearls; precious, semi-preci...",1,0.01,1.85,0.87


## Root Cause Analysis: Trade Deficit Drivers

### Key Observation:
The primary "deficit-makers" are countries whose trade with Switzerland is almost exclusively driven by precious stones and metals (HS 71). Trade in other commodity categories remains statistically insignificant for these specific trade corridors.

### Methodological Next Steps:
Due to the high concentration within this sector, the current level of data aggregation (HS2) is insufficient for a complete diagnostic. To provide actionable insights, the analysis will now proceed to:

**Granular Drill-down:** Extracting a more detailed dataset focusing specifically on HS Code 71.

**Detailed Classification:** Analyzing sub-categories at the HS4 level (if needed, also HS-6) to identify the specific high-value goods (e.g., gold vs. stones) that dominate the trade balance.

**Checking if the general table `ch_2020_24` and new `comm_code_71` match together:**

In [21]:
WITH hs4 AS(
	
	SELECT
		trade_year,
   	 	trade_flow,
		ROUND((SUM(trade_value)/1000000000),2) as total_val_bln
	FROM comm_code_71.csv -- more detailed dataset, only with HS4 of commodities_code = 71 (precious stones and metals)
	GROUP BY trade_year, trade_flow
	),

hs2 AS (
	SELECT 
	   	trade_year,
		trade_flow,
		ROUND((SUM(trade_value)/1000000000),2) as total_val_bln
	FROM ch_2020_24.csv
	WHERE commodities_code = 71
	GROUP BY trade_year, trade_flow
)

SELECT
hs4.trade_year,
	hs4.trade_flow,
	hs4.total_val_bln AS hs4_precious,
	hs2.total_val_bln AS hs2_precious
FROM hs4
FULL OUTER JOIN hs2
USING(trade_flow, trade_year)

,trade_year,trade_flow,hs4_precious,hs2_precious
0,2024,Import,125.62,125.62
1,2021,Import,109.87,109.87
2,2020,Import,103.80,103.80
3,2022,Export,119.64,119.64
4,2023,Import,121.97,121.97
5,2020,Export,85.79,85.79
6,2024,Export,136.94,136.94
7,2021,Export,105.75,105.75
8,2022,Import,117.70,117.70
9,2023,Export,126.98,126.98


**What types of precious stones and metals are covering at least 50% of the total import value from these countries?**

In [22]:
WITH all_comm AS
	(
	SELECT
		trade_year,
		partner_code,
		ROUND((SUM(trade_value)/1000000),2) AS total_trade_value_mln
	FROM ch_2020_24.csv
	GROUP BY trade_year, partner_code
	)

SELECT
	a.trade_year,
	b.partner_name,
	a.commodities_code,
	c.commodities_name,
	ROUND((SUM(a.trade_value)/1000000),2) AS precious_metals_trade_value_mln,
	MAX(all_comm.total_trade_value_mln) as total_import_value_mln, -- using aggregation MAX (while there is only one number per country per year) to not use this field in GROUP BY
	ROUND(((SUM(a.trade_value)/1000000) *100 / MAX(all_comm.total_trade_value_mln)), 2) AS percent_total_import_value,
	RANK() OVER(PARTITION BY a.trade_year, b.partner_name ORDER BY SUM(a.trade_value) DESC) AS rank_metals
FROM comm_code_71.csv AS a
	LEFT JOIN all_comm
		ON a.trade_year = all_comm.trade_year
			AND a.partner_code = all_comm.partner_code
LEFT JOIN countries.csv AS b
		ON a.partner_code = b.partner_code
LEFT JOIN commodities.csv AS c
		ON a.commodities_code = c.commodities_code
WHERE partner_name IN
	(
	SELECT partner_name
	FROM 
	-- subquery 'deficit-makers' - subset with countries, with which Switzerland had the biggest deficit on average year (at least 80% of the total trade value) and with whom the overall yearly trade value was bigger or equal to 0.1bln $ 
		(
			SELECT *
			FROM avg_agg_imports_exports
			WHERE avg_volume_year_bln >= 0.1
				AND percent_surplus_deficit_to_volume <= -80
		)
	)
	AND a.trade_flow = 'Import'
GROUP BY a.trade_year, b.partner_name, a.commodities_code, c.commodities_name
	HAVING percent_total_import_value > 50
ORDER BY partner_name, a.trade_year, rank_metals

,trade_year,partner_name,commodities_code,commodities_name,precious_metals_trade_value_mln,total_import_value_mln,percent_total_import_value,rank_metals
0,2020,Burkina Faso,7108,Gold (including gold plated with platinum) unw...,3235.14,3242.62,99.77,1
1,2021,Burkina Faso,7108,Gold (including gold plated with platinum) unw...,3378.19,3383.73,99.84,1
2,2022,Burkina Faso,7108,Gold (including gold plated with platinum) unw...,3023.26,3032.07,99.71,1
3,2023,Burkina Faso,7108,Gold (including gold plated with platinum) unw...,2982.69,2988.35,99.81,1
4,2024,Burkina Faso,7108,Gold (including gold plated with platinum) unw...,3157.50,3166.37,99.72,1
...,...,...,...,...,...,...,...,...
77,2024,United Rep. of Tanzania,7108,Gold (including gold plated with platinum) unw...,212.60,273.79,77.65,1
78,2021,Uzbekistan,7108,Gold (including gold plated with platinum) unw...,2355.45,2483.26,94.85,1
79,2022,Uzbekistan,7108,Gold (including gold plated with platinum) unw...,4274.95,4385.99,97.47,1
80,2023,Uzbekistan,7108,Gold (including gold plated with platinum) unw...,8173.83,8396.44,97.35,1


## Specific Trade Corridors

### Key Finding:
Unilateral Trade Flows Data analysis confirms that for the selected group of countries, Gold (HS 7108) constitutes 85% to 100% of their total exports to Switzerland (with Uzbekistan 2020 as the sole exception). These trade relationships are characterized by:

**Sector Concentration:** Gold is often the exclusive commodity traded.

**Trade Asymmetry:** These flows are almost entirely one-way; Switzerland acts as a pure importer with very few of exports to these nations.

### Identified High-Concentration Partners:

Burkina Faso, Dominican Republic, Ghana, Ivory Coast, Mali, Peru, Suriname, Uzbekistan.

## Analytical Drill-down:
To determine the industrial or financial nature of these imports, the dataset was expanded to the HS6 level. This allows for a granular distinction between the following sub-categories:

7108.11 – _Powder (non-monetary): Fine gold particles obtained by mechanical or chemical processes; used in electronics, dentistry, and specialized decorative coatings._


7108.12 – _Other unwrought forms (non-monetary): Investment and industrial gold in primary forms like bars, ingots, bullion, nuggets, and dore (unrefined gold)._


7108.13 – _Other semi-manufactured forms (non-monetary): Gold worked into intermediate products such as plates, sheets, strips, wire, tubes, and gold leaf; e.g. for jewellery purposes_


7108.20 – _Monetary gold: Gold held by official monetary authorities (central banks) or international financial institutions as part of their reserve assets._ 

In [21]:
WITH all_comm AS
	(
	SELECT
		trade_year,
		partner_code,
		ROUND((SUM(trade_value)/1000000),2) AS total_trade_value_mln
	FROM ch_2020_24.csv
	GROUP BY trade_year, partner_code
	)

SELECT
	a.trade_year,
	b.partner_name,
	a.commodities_code,
	ROUND((SUM(CASE
		WHEN a.trade_flow = 'Import'
		THEN a.trade_value ELSE 0 END)/1000000),2) AS gold_import_value_mln,
	MAX(all_comm.total_trade_value_mln) as total_trade_value_mln, -- using MAX not to use this field in GROUP BY
	ROUND((SUM(CASE
		WHEN a.trade_flow = 'Import'
		THEN a.trade_value
		ELSE 0 END)/1000000) *100 / MAX(all_comm.total_trade_value_mln), 2) AS percent_total_import_value
FROM comm_code_7108.csv AS a
	LEFT JOIN all_comm
		ON a.trade_year = all_comm.trade_year
			AND a.partner_code = all_comm.partner_code
LEFT JOIN countries.csv AS b
		ON a.partner_code = b.partner_code
LEFT JOIN commodities.csv AS c
		ON a.commodities_code = c.commodities_code
WHERE partner_name IN
	(
	SELECT partner_name
	FROM 
	-- subquery 'deficit-makers' - (subset) with countries, with which Switzerland had the biggest deficit on average year (at least 90% of the total trade value) and with whom the overall yearly trade value was bigger or equal to 1bln $ 
		(
		SELECT *
		FROM avg_agg_imports_exports
		WHERE avg_volume_year_bln >= 1
			AND deficit_to_volume_percent <= -85
		)
	)
GROUP BY a.trade_year, b.partner_name, a.commodities_code
	HAVING percent_total_import_value > 50
ORDER BY partner_name, a.trade_year

Error: Binder Error: Referenced column "deficit_to_volume_percent" not found in FROM clause!
Candidate bindings: "percent_surplus_deficit_to_volume", "trade_value", "avg_surplus_deficit_year_bln", "total_trade_value_mln", "commodities_name"

LINE 40: 			AND deficit_to_volume_percent <= -85
             ^

## Single Sub-category concentration

**Granular data filtering reveals that gold imports from the identified nations are exclusively concentrated within a single sub-category:**

_HS 7108.12 – Gold, unwrought (non-monetary): Includes primary forms such as bars, ingots, bullion, nuggets, and dore (unrefined gold)._

The concentration in this specific group of products indicates that Switzerland's trade with these partners is restricted to two primary functions:

**The import of finished bullion or bars as an investment** or

**import of unrefined dore gold intended for further processing and purification within Swiss refineries.**




### Sector Composition: Gold Dominance

**When breaking down the broader "Precious Metals & Stones" category (HS 71) across all Swiss trade, Gold (HS 7108) is the overwhelming driver, accounting for 84% of the total:**

In [22]:
WITH breakdown_71 AS
	(
	SELECT
		trade_year,
		trade_flow,
		commodities_code,
		ROUND((SUM(trade_value)/1000000),2) AS total_trade_value_mln
	FROM comm_code_71.csv
	GROUP BY trade_year, trade_flow, commodities_code
	),

code_71 AS (
	SELECT
		trade_year,
		trade_flow,
		ROUND((SUM(trade_value)/1000000),2) AS total_trade_value_mln_71
	FROM ch_2020_24.csv
	WHERE commodities_code = 71
	GROUP BY trade_year, trade_flow
)


SELECT
	trade_flow,
	a.commodities_code AS commodities_code,
	c.commodities_name AS precious_stone_metal,
	AVG(a.total_trade_value_mln) as avg_trade_value_mln,
	AVG(code_71.total_trade_value_mln_71) AS avg_trade_value_71_mln,
	ROUND((AVG(a.total_trade_value_mln) *100 / AVG(code_71.total_trade_value_mln_71)),0) AS percentage
FROM breakdown_71 AS a
LEFT JOIN code_71
	USING (trade_year, trade_flow)
LEFT JOIN commodities.csv AS c
		ON a.commodities_code = c.commodities_code
GROUP BY a.trade_flow, a.commodities_code, precious_stone_metal
	HAVING percentage > 0
ORDER BY a.trade_flow, avg_trade_value_mln DESC

,trade_flow,commodities_code,precious_stone_metal,avg_trade_value_mln,avg_trade_value_71_mln,percentage
0,Export,7108,Gold (including gold plated with platinum) unw...,96525.100,115021.408,84.0
1,Export,7113,"Jewellery articles and parts thereof, of preci...",11898.154,115021.408,10.0
2,Export,7110,Platinum; unwrought or in semi-manufactured fo...,1837.842,115021.408,2.0
3,Export,7106,Silver (including silver plated with gold or p...,1618.828,115021.408,1.0
4,Export,7102,"Diamonds, whether or not worked, but not mount...",1465.306,115021.408,1.0
5,Export,7112,Waste and scrap of precious metal or of metal ...,778.170,115021.408,1.0
6,Export,7103,Precious (excluding diamond) and semi-precious...,670.256,115021.408,1.0
7,Import,7108,Gold (including gold plated with platinum) unw...,97686.742,115794.906,84.0
8,Import,7113,"Jewellery articles and parts thereof, of preci...",8761.152,115794.906,8.0
9,Import,7110,Platinum; unwrought or in semi-manufactured fo...,2729.220,115794.906,2.0


# Swiss deficit

## Significant "Deficit-makers"

**Having identified gold as the primary driver of trade imbalances, we will now analyze these monoproduct economies within the broader context of all Swiss deficit-making partners.**

**The following analysis focuses on a group of 53 countries that meet two criteria:**

- Switzerland maintains a bilateral **trade deficit** with the nation.

- **The annual trade volume exceeds $100 million** (to filter out statistically insignificant outliers).

**To evaluate the impact of this group, I address 3 key questions:**

**- Trade Volume: What is the average annual trade value with these partners?**

**- Deficit Scale: How big deficit do they create?**

**- Gold Concentration: What specific role does gold play in these calculations?**

In [9]:
-- AVG of import/export by partner counted as SUM divided by 5y to simplify the code. 
WITH imports_avg_year AS
		(
		SELECT
			partner_code,
			ROUND(((SUM(trade_value)/5)/1000000000),2) AS avg_import_bln
		FROM ch_2020_24.csv
		WHERE trade_flow = 'Import'
		GROUP BY partner_code
		),

exports_avg_year AS
		(
		SELECT
			partner_code,
			ROUND(((SUM(trade_value)/5)/1000000000),2) AS avg_export_bln
		FROM ch_2020_24.csv
		WHERE trade_flow = 'Export'
		GROUP BY partner_code
		),

-- HS-4 code "7108" means goods made out of gold. AVG as SUM divided by 5y to simplify the code. 
ch_gold AS
		(
		SELECT
			partner_code,
			ROUND(((SUM(trade_value)/5)/1000000000),2) AS avg_gold_value_bln
		FROM comm_code_7108.csv
		WHERE trade_flow = 'Import'
		GROUP BY partner_code
		),

-- HS-2 code "71" means precious stones and metals. AVG as SUM divided by 5y to simplify the code. 
ch_precious_metals AS
		(
		SELECT
			partner_code,
			ROUND(((SUM(trade_value)/5)/1000000000),2) AS avg_precious_stones_metals_value_bln
		FROM ch_2020_24.csv
		WHERE commodities_code = 71 AND trade_flow = 'Import'
		GROUP BY partner_code
		)
	
SELECT
	c.partner_name,
	gold.avg_gold_value_bln AS avg_gold_import_value_bln,
	pm.avg_precious_stones_metals_value_bln AS avg_precious_stones_metals_import_value_bln,
	i.avg_import_bln,
	avg_export_bln + avg_import_bln AS avg_volume_year_bln,
	avg_export_bln - avg_import_bln AS avg_deficit_year_bln,
	ROUND(((avg_export_bln - avg_import_bln) * 100 / (avg_export_bln + avg_import_bln)),1) AS deficit_to_volume_percent,
	ROUND((gold.avg_gold_value_bln * 100 / i.avg_import_bln),1) AS imported_gold_to_import_percent
FROM exports_avg_year as e
LEFT JOIN imports_avg_year as i
	ON e.partner_code = i.partner_code
LEFT JOIN ch_gold AS gold
	ON i.partner_code = gold.partner_code
LEFT JOIN ch_precious_metals as pm
	ON i.partner_code = pm.partner_code
LEFT JOIN countries.csv AS c
	ON i.partner_code = c.partner_code
-- filter the countries with which Switzerland had, on average, a negative balance in trade and for which the yearly volume is bigger than 0.1 billion USD (to exclude outliers with a very small impact).
WHERE avg_deficit_year_bln < 0 AND avg_volume_year_bln > 0.1
ORDER BY deficit_to_volume_percent

,partner_name,avg_gold_import_value_bln,avg_precious_stones_metals_import_value_bln,avg_import_bln,avg_volume_year_bln,avg_deficit_year_bln,deficit_to_volume_percent,imported_gold_to_import_percent
0,Liberia,0.52,0.52,0.52,0.52,-0.52,-100.0,100.0
1,Suriname,1.19,1.19,1.19,1.19,-1.19,-100.0,100.0
2,Burkina Faso,3.16,3.16,3.16,3.17,-3.15,-99.4,100.0
3,Mali,1.05,1.05,1.05,1.06,-1.04,-98.1,100.0
4,Kyrgyzstan,0.87,0.87,0.87,0.88,-0.86,-97.7,100.0
5,Ghana,3.22,3.22,3.32,3.36,-3.28,-97.6,97.0
6,Mauritania,0.37,0.37,0.37,0.38,-0.36,-94.7,100.0
7,Uzbekistan,4.19,4.20,4.20,4.32,-4.08,-94.4,99.8
8,Ivory Coast,1.53,1.53,1.58,1.63,-1.53,-93.9,96.8
9,Nicaragua,0.21,0.21,0.26,0.27,-0.25,-92.6,80.8


**How big is Switzerland’s trade deficit with these countries?**

These filtered countries make 99.7 % of the overall deficit (without states with <100 mln USD in bilateral trade)

In [61]:
SELECT SUM(avg_deficit_year_bln) AS filtered_deficit_total,
		COUNT(partner_name) AS count
FROM avg_agg_imports_exports_gold

,filtered_deficit_total,count
0,-64.21,53


### Major Deficit Drivers with a deficit to trade volume of 90%+


**Disproportionate impact of the Top 13 partners (those with a 90%+ deficit-to-trade ratio) relative to the broader group of deficit-making countries:**

In [63]:
SELECT
	SUM(avg_deficit_year_bln) AS deficit_top_deficit_makers_bln,
	ROUND((SUM(avg_deficit_year_bln)/64.43) * (-100),1) AS percent_of_total_deficit,
	COUNT(partner_name) AS count_countries
FROM avg_agg_imports_exports_gold
WHERE deficit_to_volume_percent < -90

,deficit_top_deficit_makers_bln,percent_of_total_deficit,count_countries
0,-17.98,27.9,13


**A list of these 13 countries with data showing how big part of their export to Switzerland consist of gold:**

In [10]:
SELECT
	*
FROM avg_agg_imports_exports_gold
WHERE deficit_to_volume_percent < -90

,partner_name,avg_gold_import_value_bln,avg_precious_stones_metals_import_value_bln,avg_import_bln,avg_volume_year_bln,avg_deficit_year_bln,deficit_to_volume_percent,imported_gold_to_import_percent
0,Liberia,0.52,0.52,0.52,0.52,-0.52,-100.0,100.0
1,Suriname,1.19,1.19,1.19,1.19,-1.19,-100.0,100.0
2,Burkina Faso,3.16,3.16,3.16,3.17,-3.15,-99.4,100.0
3,Mali,1.05,1.05,1.05,1.06,-1.04,-98.1,100.0
4,Kyrgyzstan,0.87,0.87,0.87,0.88,-0.86,-97.7,100.0
5,Ghana,3.22,3.22,3.32,3.36,-3.28,-97.6,97.0
6,Mauritania,0.37,0.37,0.37,0.38,-0.36,-94.7,100.0
7,Uzbekistan,4.19,4.20,4.20,4.32,-4.08,-94.4,99.8
8,Ivory Coast,1.53,1.53,1.58,1.63,-1.53,-93.9,96.8
9,Nicaragua,0.21,0.21,0.26,0.27,-0.25,-92.6,80.8


## Extreme Trade Imbalance: The One-Way Corridors

**Key Finding:** Structural Asymmetry Switzerland has **13 trading partners where trade is almost entirely one-way.** In these specific cases, the trade deficit accounts for **over 90% of the total bilateral trade value.**

**Macro Impact:** Despite being a small group, these 13 countries are responsible for nearly **30% of Switzerland's total global trade deficit.**

**The Gold Factor:** Gold represents virtually the entire export value to Switzerland from these countries.

- **Near-Total Concentration:** For 11 of these countries, gold accounts for **97% to 100%** of all goods sold to Switzerland.

- **Minor Exceptions:** The Dominican Republic (**93%**) and Nicaragua (**81%**) are the only partners in this group where other goods have any measurable presence, though gold remains the overwhelming driver.


### Commodity monoculture

Dozens of Switzerland’s partners exhibit a commodity monoculture, where gold is the near-exclusive export. The same countries are also responsible for the largest bilateral trade deficits.

**What happens to the deficit profile if we apply "Gold Concentration" as our primary filter across the entire group of 53 deficit-making nations?**

In [64]:
SELECT
	CASE WHEN imported_gold_to_import_percent >= 80 THEN '80-100'
		ELSE '0-79' END AS percent_imported_gold_to_import,
	COUNT(partner_name) AS num_countries,
	SUM(avg_deficit_year_bln) AS sum_deficit_per_year_bln,
	ROUND((SUM(avg_deficit_year_bln)/64.43) * (-100),0) AS percent_of_total_deficit,
	SUM(avg_gold_import_value_bln) AS gold_import_value_per_year_bln
FROM avg_agg_imports_exports_gold
GROUP BY percent_imported_gold_to_import
ORDER BY percent_imported_gold_to_import DESC

,percent_imported_gold_to_import,num_countries,sum_deficit_per_year_bln,percent_of_total_deficit,gold_import_value_per_year_bln
0,80-100,27,-34.84,54.0,47.94
1,0-79,26,-29.37,46.0,10.64


**Table shows the gold import in relation to the total import during an average year:**

In [71]:
WITH import AS (
			SELECT
			partner_code,
			ROUND((SUM(trade_value)/5/1000000000),2) as avg_import_value_bln_usd
		FROM ch_2020_24.csv AS a
		WHERE trade_flow = 'Import'
		GROUP BY partner_code
	)

SELECT
	SUM(avg_gold_value_bln) AS total_import_gold_year,
	SUM(import.avg_import_value_bln_usd) AS yearly_import_value_bln_usd,
	ROUND((SUM(avg_gold_value_bln)*100/SUM(import.avg_import_value_bln_usd)),0) AS gold_import_percent
	FROM(
		SELECT
			partner_code,
			ROUND(((SUM(trade_value)/5)/1000000000),2) AS avg_gold_value_bln
		FROM comm_code_7108.csv
		WHERE trade_flow = 'Import'
		GROUP BY partner_code) AS gold_per_partner
	LEFT JOIN import
		USING (partner_code)

,total_import_gold_year,yearly_import_value_bln_usd,gold_import_percent
0,97.66,327.05,30.0


### Context for gold: Top 5 Import Partners

**Swiss Perspective on Switzerland’s five largest import sources and their respective trade surplus or deficit (for wider context):**

In [75]:
-- AVG of import/export by partner counted as SUM divided by 5y to simplify the code. 
WITH imports_avg_year AS
		(
		SELECT
			partner_code,
			ROUND(((SUM(trade_value)/5)/1000000000),2) AS avg_import_bln
		FROM ch_2020_24.csv
		WHERE trade_flow = 'Import'
		GROUP BY partner_code
		),

exports_avg_year AS
		(
		SELECT
			partner_code,
			ROUND(((SUM(trade_value)/5)/1000000000),2) AS avg_export_bln
		FROM ch_2020_24.csv
		WHERE trade_flow = 'Export'
		GROUP BY partner_code
		)
	
SELECT
	c.partner_name,
	i.avg_import_bln,
	avg_export_bln + avg_import_bln AS avg_volume_year_bln,
	avg_export_bln - avg_import_bln AS avg_deficit_year_bln,
	ROUND(((avg_export_bln - avg_import_bln) * 100 / (avg_export_bln + avg_import_bln)),1) AS deficit_to_volume_percent
FROM exports_avg_year as e
LEFT JOIN imports_avg_year as i
	ON e.partner_code = i.partner_code
LEFT JOIN countries.csv AS c
	ON i.partner_code = c.partner_code
ORDER BY avg_import_bln DESC
LIMIT 5

,partner_name,avg_import_bln,avg_volume_year_bln,avg_deficit_year_bln,deficit_to_volume_percent
0,Germany,64.55,116.54,-12.56,-10.8
1,USA,29.54,97.06,37.98,39.1
2,Italy,26.26,49.43,-3.09,-6.3
3,France,20.42,40.01,-0.83,-2.1
4,China,19.96,56.38,16.46,29.2


### Gold 27
**These 27 monoproduct exporters are responsible for $48 billion of Switzerland’s trade deficit:**

In [52]:
SELECT
	partner_name,
	avg_gold_import_value_bln,
	avg_precious_stones_metals_import_value_bln,
	avg_import_bln,
	avg_volume_year_bln,
	avg_deficit_year_bln,
	deficit_to_volume_percent,
	imported_gold_to_import_percent
FROM avg_agg_imports_exports_gold
WHERE  imported_gold_to_import_percent >= 80
	AND imported_gold_to_import_percent IS NOT NULL

,partner_name,avg_gold_import_value_bln,avg_precious_stones_metals_import_value_bln,avg_import_bln,avg_volume_year_bln,avg_deficit_year_bln,deficit_to_volume_percent,imported_gold_to_import_percent
0,Suriname,1.19,1.19,1.19,1.19,-1.19,-100.0,100.0
1,Liberia,0.52,0.52,0.52,0.52,-0.52,-100.0,100.0
2,Burkina Faso,3.16,3.16,3.16,3.17,-3.15,-99.4,100.0
3,Mali,1.05,1.05,1.05,1.06,-1.04,-98.1,100.0
4,Kyrgyzstan,0.87,0.87,0.87,0.88,-0.86,-97.7,100.0
5,Ghana,3.22,3.22,3.32,3.36,-3.28,-97.6,97.0
6,Mauritania,0.37,0.37,0.37,0.38,-0.36,-94.7,100.0
7,Uzbekistan,4.19,4.20,4.20,4.32,-4.08,-94.4,99.8
8,Ivory Coast,1.53,1.53,1.58,1.63,-1.53,-93.9,96.8
9,Nicaragua,0.21,0.21,0.26,0.27,-0.25,-92.6,80.8


## The Impact of High-Concentration Gold Exporters

**The "Gold 27" Group:** More than half of all "deficit-makers" (27 out of 53 countries) focus almost exclusively on gold. For these nations, gold makes **over 80% of their exports** to Switzerland.

## Key Figures:

**Deficit Contribution:** This group is responsible for **54% of Switzerland's total trade deficit** (34.84 billion USD).

**Gold Supply:** These 27 countries supply roughly **50% of all gold imported to Switzerland** - nearly 48 billion USD out of a total 97.7 billion USD.

**A Surprising Perspective:** Gold from these 27 countries alone accounts for **15% of Switzerland’s total imports.** To put this in perspective, this single commodity from this specific group of countries is worth more than **all goods imported from the USA**, Switzerland’s second-largest trading partner.

# Wider context: Gold trade

**How big is role of Swiss gold trade compared to the whole world?**

In [3]:
WITH world AS (
SELECT
	trade_year,
	trade_flow,
	ROUND((SUM(trade_value)/1000000000),2) AS total_trade_value_bln
FROM 'world_gold.csv'
GROUP BY trade_year, trade_flow
ORDER BY trade_year, trade_flow
),
switzerland AS(
SELECT
	trade_year,
	trade_flow,
	ROUND((SUM(trade_value)/1000000000),2) AS swiss_total_trade_value_bln
FROM 'comm_code_7108.csv'
GROUP BY trade_year, trade_flow
ORDER BY trade_year, trade_flow
	)
	
SELECT
	world.trade_year,
	world.trade_flow,
	swiss_total_trade_value_bln,
	total_trade_value_bln,
	ROUND((swiss_total_trade_value_bln *100 /total_trade_value_bln),1) AS percent
FROM world
INNER JOIN switzerland
	USING(trade_year, trade_flow)
ORDER BY trade_year, trade_flow

,trade_year,trade_flow,swiss_total_trade_value_bln,total_trade_value_bln,percent
0,2020,Export,71.66,385.87,18.6
1,2020,Import,87.93,402.84,21.8
2,2021,Export,86.76,399.19,21.7
3,2021,Import,92.69,438.78,21.1
4,2022,Export,100.23,444.30,22.6
5,2022,Import,98.95,485.53,20.4
6,2023,Export,107.17,485.42,22.1
7,2023,Import,102.25,551.02,18.6
8,2024,Export,116.82,503.17,23.2
9,2024,Import,106.61,557.82,19.1


**Who is number one in gold trading and how big is its yearly gold trade?**

In [4]:
WITH world AS (
SELECT
	trade_year,
	trade_flow,
	reporter_code,
	ROUND((SUM(trade_value)/1000000000),2) AS total_trade_value_bln
FROM 'world_gold.csv'
GROUP BY trade_year, trade_flow, reporter_code
)
	
SELECT *
FROM (
	SELECT
		trade_flow,
		partner_name,
		ROUND(((SUM(trade_value)/5)/1000000000),2) AS avg_gold_trade_value_bln,
		RANK() OVER(PARTITION BY trade_flow ORDER BY (SUM(trade_value)/5) DESC) as rank
	FROM world_gold.csv AS a
	LEFT JOIN countries.csv AS b
		ON a.reporter_code = b.partner_code
	GROUP BY trade_flow, partner_name
	ORDER BY rank
)
WHERE rank<=10

,trade_flow,partner_name,avg_gold_trade_value_bln,rank
0,Export,Switzerland,96.53,1
1,Import,Switzerland,97.69,1
2,Export,United Kingdom,53.24,2
3,Import,China,65.99,2
4,Export,"China, Hong Kong SAR",39.80,3
5,Import,United Kingdom,61.70,3
6,Export,United Arab Emirates,30.31,4
7,Import,United Arab Emirates,44.34,4
8,Export,USA,28.24,5
9,Import,India,42.90,5


**List of countries, their gold trade with Switzerland and total value of gold trade by year.**

Query takes records from broken down category 7108. It appears that 710812 is the most traded among gold goods.

_HS-4 code "7108" means goods made out of gold._

_HS-6 code "710812" means gold in other unwrought forms (non-monetary): Investment and industrial gold in primary forms like bars, ingots, bullion, nuggets, and dore (unrefined gold)._

In [8]:
SELECT
	partner_name,
	commodities_code,
	ROUND(((SUM(trade_value)/5)/1000000000),2) AS gold_trade_value_bln
FROM comm_code_7108.csv
LEFT JOIN countries.csv
	USING (partner_code)
GROUP BY partner_name, commodities_code
HAVING gold_trade_value_bln > 0.1
	ORDER BY gold_trade_value_bln desc 


,partner_name,commodities_code,gold_trade_value_bln
0,USA,710812,24.07
1,China,710812,19.57
2,India,710812,17.17
3,United Kingdom,710812,16.43
4,United Arab Emirates,710812,12.15
...,...,...,...
65,Norway,710812,0.13
66,Cambodia,710812,0.13
67,Singapore,710813,0.12
68,Nigeria,710812,0.11


# Executive Summary: Switzerland as the Global Gold Hub

## 1. The "Big Picture" Context

Switzerland is the undisputed leader in the global gold industry. According to UN data, the country controls 20% of the world’s gold trade, trading an average of $100 billion in imports and exports annually.

- **Quality Standards:** Switzerland’s influence is cemented by its refining capacity. 3 out of the 7 global LBMA Referees (the organizations that set world quality standards) are Swiss-based.

- **The Business Model:** This is a "Refine & Re-export" model. Switzerland imports raw gold (bars, bullion, or dore), purifies it to the highest standards, and sells it back to the global market.

## 2. Understanding the "Artificial" Deficit

The trade deficit with certain nations is not a sign of economic weakness. Instead, it is a structural result of Switzerland’s role in the supply chain.

- **The "Gold 27" Impact:** When we look at countries where gold makes up over 80% of their exports to Switzerland, we find a group of 27 states. This group alone is responsible for a $34.84 billion deficit.

- **Investment Focus:** This analysis focuses strictly on HS 7108.12 (gold for investment/refining), excluding jewellery or recycled scrap, which makes the concentration even more striking.

##  3. Strategic Dependency

### The data reveals a critical insight:

- **50% of all gold imported into Switzerland comes from these 27 monoproduct exporters.**

- To fulfil global demand for refined "Swiss-made" gold, the country has become strategically dependent on these 27 partners. The deficit is simply the "price" of maintaining its position as the world's gold refinery.

## Important Note:
**Value vs. Volume** It is important to note that these figures represent **trade value in USD, not physical weight**. While Switzerland’s total gold imports and exports are nearly equal in dollar terms, this does not mean the physical amount of gold is the same. Swiss refineries profit by "adding value" - transforming raw or unrefined gold into high-purity products that sell at a premium on the global market.

_1. https://www.swissinfo.ch/eng/business/switzerland-the-world-s-gold-hub/33706126_

_2. https://www.lbma.org.uk/articles/lbma-announces-new-referees-of-the-good-delivery-programme_

_3. LBMA: London Bullion Market Association_